## Project luther 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re


## Scraping list of authors

In [2]:
response = requests.get('http://www.gutenberg.org/browse/languages/es')
soup_authors = BeautifulSoup(response.text, "lxml")

In [3]:
authors = soup_authors.find_all("h2")

In [4]:
books_authors = soup_authors.find("div", class_="pgdbbylanguage").find_all("a")[1:]
books_authors = [[str(entry.renderContents().decode(encoding='UTF-8')), str(entry) ] 
                 if len(entry.renderContents().decode(encoding='UTF-8'))!= 0
                 else ["END","END"] 
                for entry in books_authors]

books_authors

[['About, Edmond, 1828-1885',
  '<a href="/browse/authors/a#a5182">About, Edmond, 1828-1885</a>'],
 ['Germana', '<a href="/ebooks/29640">Germana</a>'],
 ['La nariz de un notario',
  '<a href="/ebooks/26404">La nariz de un notario</a>'],
 ['Real Academia de la Historia (Spain)',
  '<a href="#a47252">Real Academia de la Historia (Spain)</a>'],
 ['END', 'END'],
 ['Acosta, José de, 1540-1600',
  '<a href="/browse/authors/a#a45903">Acosta, José de, 1540-1600</a>'],
 ['Historia natural y moral de las Indias (vol 2 of 2)',
  '<a href="/ebooks/50430">Historia natural y moral de las Indias (vol 2 of 2)</a>'],
 ['END', 'END'],
 ['Aesop, 620? BC-564? BC',
  '<a href="/browse/authors/a#a18">Aesop, 620? BC-564? BC</a>'],
 ['Las Fábulas de Esopo, Vol. 01',
  '<a href="/ebooks/21143">Las Fábulas de Esopo, Vol. 01</a>'],
 ['Las Fábulas de Esopo, Vol. 02',
  '<a href="/ebooks/20029">Las Fábulas de Esopo, Vol. 02</a>'],
 ['Las Fábulas de Esopo, Vol. 03',
  '<a href="/ebooks/21144">Las Fábulas de Esopo, 

Making a list with authors and books. Identifying authors and identification numbers that will be used in the links for downloading each book.

In [5]:
books_ordered = []
for entry in books_authors:
    if entry[1].find("ebooks") != -1:
        books_ordered.append([entry[0], entry[1].split("ebooks/")[1].split("\"")[0]])
    elif entry[1].find("authors") != -1:
        books_ordered.append([entry[0], "author"])
    else:
        pass

books_ordered


[['About, Edmond, 1828-1885', 'author'],
 ['Germana', '29640'],
 ['La nariz de un notario', '26404'],
 ['Acosta, José de, 1540-1600', 'author'],
 ['Historia natural y moral de las Indias (vol 2 of 2)', '50430'],
 ['Aesop, 620? BC-564? BC', 'author'],
 ['Las Fábulas de Esopo, Vol. 01', '21143'],
 ['Las Fábulas de Esopo, Vol. 02', '20029'],
 ['Las Fábulas de Esopo, Vol. 03', '21144'],
 ['Aguado, Pedro de, active 16th century', 'author'],
 ['Historia de Venezuela, Tomo I', '39947'],
 ['Historia de Venezuela, Tomo II', '39948'],
 ['Aguinaldo, Emilio, 1869-1964', 'author'],
 ['Reseña Veridica de la Revolución Filipina', '14307'],
 ['Aimard, Gustave, 1818-1883', 'author'],
 ['Las noches mejicanas', '54430'],
 ['Los Merodeadores de Fronteras', '46279'],
 ['Alarcón, Pedro Antonio de, 1833-1891', 'author'],
 ['Novelas Cortas', '15532'],
 ['El Capitán Veneno<br/>The Hispanic Series', '29731'],
 ['Novelas Cortas', '15532'],
 ['El sombrero de tres picos\r<br/>Historia verdadera de un sucedido que 

Making dictionary with authors as keys and lists of books as entries.

In [6]:
books_dictio = {}
for index in range(len(books_ordered)):
    if books_ordered[index][1] == "author":
        books_dictio[books_ordered[index][0]] = []
        key = books_ordered[index][0]
    else:
        books_dictio[key].append([books_ordered[index][0].split("<br/>")[0], books_ordered[index][1]])
books_dictio

{'About, Edmond, 1828-1885': [['Germana', '29640'],
  ['La nariz de un notario', '26404']],
 'Acosta, José de, 1540-1600': [['Historia natural y moral de las Indias (vol 2 of 2)',
   '50430']],
 'Aesop, 620? BC-564? BC': [['Las Fábulas de Esopo, Vol. 01', '21143'],
  ['Las Fábulas de Esopo, Vol. 02', '20029'],
  ['Las Fábulas de Esopo, Vol. 03', '21144']],
 'Aguado, Pedro de, active 16th century': [['Historia de Venezuela, Tomo I',
   '39947'],
  ['Historia de Venezuela, Tomo II', '39948']],
 'Aguinaldo, Emilio, 1869-1964': [['Reseña Veridica de la Revolución Filipina',
   '14307']],
 'Aimard, Gustave, 1818-1883': [['Las noches mejicanas', '54430'],
  ['Los Merodeadores de Fronteras', '46279']],
 'Alarcón, Pedro Antonio de, 1833-1891': [['Novelas Cortas', '15532'],
  ['El Capitán Veneno', '29731'],
  ['Novelas Cortas', '15532'],
  ['El sombrero de tres picos\r', '29506'],
  ['Viajes por España', '26314']],
 'Alas, Leopoldo, 1852-1901': [['La Regenta', '17073'],
  ['Su único hijo', '173

In [7]:
author_list = []
for entry in books_ordered:
    if entry[1]=="author" and entry[0].find("19")!=-1: 
        author_list.append(entry[0])

for author in author_list:
    print(author)

Aguinaldo, Emilio, 1869-1964
Alas, Leopoldo, 1852-1901
Alvarez, Agustín, 1857-1914
Alvarez Guerra, Juan, 1843-1905
Alvarez, José S. (José Sixto), 1858-1903
Álvarez Quintero, Joaquín, 1873-1944
Álvarez Quintero, Serafín, 1871-1938
Andreyev, Leonid, 1871-1919
Aranda y Sanjuan, Manuel, 1845-1900
Argilagos, Rafael G. (Rafael Graciano), 1885-1967
Arniches y Barrera, Carlos, 1866-1943
Azaña, Manuel, 1880-1940
Bandelier, Adolph Francis Alphonse, 1840-1914
Baroja, Pío, 1872-1956
Bécker, Jerónimo, 1857-1925
Benavente, Jacinto, 1866-1954
Blasco, Cosme, 1838-1900
Blasco Ibáñez, Vicente, 1867-1928
Bodenbender, Guillermo, 1857-1941
Brau, Salvador, 1842-1912
Bunge, Carlos O. (Carlos Octavio), 1875-1918
Bunnell, Otis Gridley, 1868-1929
Byington, Steven T. (Steven Tracy), 1868-1957
Calcagno, Francisco, 1827-1903
Calderón, Sofronio G., 1878-1954?
Callejo Ferrer, Fernando, 1862-1926
Camba, Julio, 1884?-1962
Cané, Miguel, 1851-1905
Cárdenas y Echarte, Raúl de, 1884-1979
Carrere, Emilio, 1881-1947
Castro,

In [8]:
author_surnames_g = [ [author.split(",")[0], author] for author in author_list]
author_surnames_g

[['Aguinaldo', 'Aguinaldo, Emilio, 1869-1964'],
 ['Alas', 'Alas, Leopoldo, 1852-1901'],
 ['Alvarez', 'Alvarez, Agustín, 1857-1914'],
 ['Alvarez Guerra', 'Alvarez Guerra, Juan, 1843-1905'],
 ['Alvarez', 'Alvarez, José S. (José Sixto), 1858-1903'],
 ['Álvarez Quintero', 'Álvarez Quintero, Joaquín, 1873-1944'],
 ['Álvarez Quintero', 'Álvarez Quintero, Serafín, 1871-1938'],
 ['Andreyev', 'Andreyev, Leonid, 1871-1919'],
 ['Aranda y Sanjuan', 'Aranda y Sanjuan, Manuel, 1845-1900'],
 ['Argilagos', 'Argilagos, Rafael G. (Rafael Graciano), 1885-1967'],
 ['Arniches y Barrera', 'Arniches y Barrera, Carlos, 1866-1943'],
 ['Azaña', 'Azaña, Manuel, 1880-1940'],
 ['Bandelier', 'Bandelier, Adolph Francis Alphonse, 1840-1914'],
 ['Baroja', 'Baroja, Pío, 1872-1956'],
 ['Bécker', 'Bécker, Jerónimo, 1857-1925'],
 ['Benavente', 'Benavente, Jacinto, 1866-1954'],
 ['Blasco', 'Blasco, Cosme, 1838-1900'],
 ['Blasco Ibáñez', 'Blasco Ibáñez, Vicente, 1867-1928'],
 ['Bodenbender', 'Bodenbender, Guillermo, 1857-19

## Authors from wikipedia  
Downloading list of authors from the XX century.

In [9]:
url = "https://petscan.wmflabs.org/?lang=es&project=wikipedia&cats=Escritores_en_espa%C3%B1ol_del_siglo_XX&ns=0&depth=-1&max=30000&start=0&format=html&redirects=&callback="
response = requests.get(url)
soup_wiki = BeautifulSoup(response.text, "lxml")
authors_w = soup_wiki.find_all("div", class_="col-md-12")[1].find_all("a")[1:]
author_list_w = [str(author.renderContents().decode(encoding='UTF-8')) for author in authors_w ]
author_list_w

['Ana María Matute',
 'Camilo José Cela',
 'Ernesto Sabato',
 'Federico García Lorca',
 'Juan Marsé',
 'Mario Vargas Llosa',
 'Pablo Neruda',
 'Antonio Machado',
 'Pedro Salinas',
 'Vicente Aleixandre',
 'Luis Cernuda',
 'Rafael Alberti',
 'Miguel de Unamuno',
 'Ramón María del Valle-Inclán',
 'Ramón Gómez de la Serna',
 'Francisco Ayala',
 'José Díaz Fernández',
 'Joaquín Arderíus',
 'César Arconada',
 'Antonio Buero Vallejo',
 'José Hierro',
 'Ángel González',
 'Francisco Nieva',
 'Álvaro del Amo',
 'Augusto Monterroso',
 'Arturo Pérez-Reverte',
 'Gabriel García Márquez',
 'Juan Rulfo',
 'Manuel Azaña',
 'Emilio Prados',
 'Manuel Altolaguirre',
 'Gerardo Diego',
 'Dámaso Alonso',
 'Juan Eduardo Cirlot',
 'Gabriel Celaya',
 'Manuel Vázquez Montalbán',
 'Antonio Martínez Sarrión',
 'Antonio Espina',
 'Mauricio Bacarisse',
 'Francisco Villaespesa',
 'Eduardo Marquina',
 'José María Álvarez',
 'Félix de Azúa',
 'Ana María Moix',
 'Leopoldo María Panero',
 'Ramón Irigoyen',
 'Marcos Ricar

Extracting surnames.

In [10]:
author_surnames_w = [ author.split()[-1] for author in author_list_w]
author_surnames_w

['Matute',
 'Cela',
 'Sabato',
 'Lorca',
 'Marsé',
 'Llosa',
 'Neruda',
 'Machado',
 'Salinas',
 'Aleixandre',
 'Cernuda',
 'Alberti',
 'Unamuno',
 'Valle-Inclán',
 'Serna',
 'Ayala',
 'Fernández',
 'Arderíus',
 'Arconada',
 'Vallejo',
 'Hierro',
 'González',
 'Nieva',
 'Amo',
 'Monterroso',
 'Pérez-Reverte',
 'Márquez',
 'Rulfo',
 'Azaña',
 'Prados',
 'Altolaguirre',
 'Diego',
 'Alonso',
 'Cirlot',
 'Celaya',
 'Montalbán',
 'Sarrión',
 'Espina',
 'Bacarisse',
 'Villaespesa',
 'Marquina',
 'Álvarez',
 'Azúa',
 'Moix',
 'Panero',
 'Irigoyen',
 'Barnatán',
 'Colinas',
 'Talens',
 'Siles',
 'Andreu',
 'Inglada',
 'Becerra',
 'Llamazares',
 'Valverde',
 'Montero',
 'Ballester',
 'Cunqueiro',
 'Perucho',
 'Santos',
 'Alegría',
 'Allende',
 'Arguedas',
 'Asturias',
 'Benedetti',
 'Ribatto',
 'Dolina',
 'Paz',
 'Cortázar',
 'Bolaño',
 'Gallegos',
 'Gallegos',
 'Vázquez-Figueroa',
 'Luque',
 'Machado',
 'Jodorowsky',
 'León',
 'Laforet',
 'Huidobro',
 'Zambrano',
 'Bello',
 'Baquero',
 'Buesa'

Crossing list of authors from wikipedia with list of authors in spanish from gutenberg.org

In [11]:
final_authors = []
for author_w in author_surnames_w:
    for author_g in author_surnames_g:
        if author_w == author_g[0]:
            final_authors.append(author_g[1])
set(final_authors)

{'Azaña, Manuel, 1880-1940',
 'Baroja, Pío, 1872-1956',
 'Benavente, Jacinto, 1866-1954',
 'Blasco, Cosme, 1838-1900',
 'Bécker, Jerónimo, 1857-1925',
 'Calderón, Sofronio G., 1878-1954?',
 'Camba, Julio, 1884?-1962',
 'Cané, Miguel, 1851-1905',
 'Carrere, Emilio, 1881-1947',
 'Castro, Cristóbal de, 1880?-1953',
 'Coloma, Luis, 1851-1915',
 'Darío, Rubén, 1867-1916',
 'Delgado, Rafael, 1853-1914',
 'Dicenta, Joaquín, 1862-1917',
 'Drago, Luis María, 1859-1921',
 'Dávalos, Juan Carlos, 1887-1959',
 'Díaz, Porfirio, 1830-1915',
 'Echegaray, José, 1832-1916',
 'Espina, Concha, 1869-1955',
 'Jiménez, Juan Ramón, 1881-1958',
 'Larreta, Enrique, 1875-1961',
 'León, Ricardo, 1877-1943',
 'López, Vicente Fidel, 1815-1903',
 'Machado, Antonio, 1875-1939',
 'Mateos, Juan A. (Juan Antonio), 1831-1913',
 'Molina, Enrique, 1871-1964',
 'Moreno, Francisco Pascasio, 1852-1919',
 'Nogales, José, 1860-1908',
 'Palma, Angélica, 1883-1935',
 'Palma, Ricardo, 1833-1919',
 'Quiroga, Horacio, 1878-1937',
 '

In [12]:
for item in final_authors:
    print(item, books_dictio[item])

Machado, Antonio, 1875-1939 [['Páginas escogidas', '49437']]
Unamuno, Miguel de, 1864-1936 [['Abel Sánchez: Una Historia de Pasión', '44512'], ['Amor y Pedagogía', '49149'], ['Niebla (Nivola)', '49836'], ['La Tía Tula (Novela)', '44358'], ['Tres novelas ejemplares y un prólogo', '55916'], ['La voz de la conseja, t.1\r', '40827']]
Valle-Inclán, Ramón del, 1866-1936 [['La media noche', '55448'], ['Romance de lobos, comedia barbara', '10506'], ['Sonata de estío: memorias del marqués de Bradomín', '42424'], ['Sonata de otoño; Sonata de invierno: memorias del Marqués de Bradomín', '46182'], ['Sonata de primavera: memorias del marqués de Bradomín', '42440']]
Azaña, Manuel, 1880-1940 [['La Biblia en España, Tomo I (de 3)\r', '51019'], ['La Biblia en España, Tomo II (de 3)', '51020'], ['La Biblia en España, Tomo III (de 3)', '51689']]
Espina, Concha, 1869-1955 [['Agua de Nieve (Novela)', '47388'], ['Despertar Para Morir (Novela)', '48954'], ['Dulce Nombre (Novela)', '44529'], ['La Esfinge Mara

Selecting a some authors.

In [13]:
selected_authors = ['Unamuno, Miguel de, 1864-1936','Valle-Inclán, Ramón del, 1866-1936', 
                    'Baroja, Pío, 1872-1956', 'Benavente, Jacinto, 1866-1954']
for item in selected_authors:
    print(item, books_dictio[item])

Unamuno, Miguel de, 1864-1936 [['Abel Sánchez: Una Historia de Pasión', '44512'], ['Amor y Pedagogía', '49149'], ['Niebla (Nivola)', '49836'], ['La Tía Tula (Novela)', '44358'], ['Tres novelas ejemplares y un prólogo', '55916'], ['La voz de la conseja, t.1\r', '40827']]
Valle-Inclán, Ramón del, 1866-1936 [['La media noche', '55448'], ['Romance de lobos, comedia barbara', '10506'], ['Sonata de estío: memorias del marqués de Bradomín', '42424'], ['Sonata de otoño; Sonata de invierno: memorias del Marqués de Bradomín', '46182'], ['Sonata de primavera: memorias del marqués de Bradomín', '42440']]
Baroja, Pío, 1872-1956 [['El aprendiz de conspirador', '47103'], ['Los Caminos del Mundo', '49280'], ['Los Caudillos de 1830', '53517'], ['Con la Pluma y con el Sable: Crónica de 1820 a 1823', '49470'], ['Los Contrastes de la Vida', '51858'], ['El Escuadrón del Brigante', '48783'], ['Las Furias', '55157'], ['Las inquietudes de Shanti Andia', '12848'], ['Las inquietudes de Shanti Andía', '47057'], 

### Downloading all Books from selected authors

In [ ]:
for author in selected_authors[-1:]:
    for index, book in enumerate(books_dictio[author]):
        command = "wget -w 2 -m -H http://www.gutenberg.org/files/" + book[1] + "/"
        os.system(command)
        time.sleep(60)


### Renaming files

In [20]:
file_code = []
for author in final_authors:
    for index, book in enumerate(books_dictio[author]):
        name_author = "_".join([author.split(",")[1][1:], author.split(",")[0], book[0]])
        remove = [':', ' ', ';', '\,', '\.', '-']
        for item in remove:
            name_author = re.sub(item, '_', name_author)
        name_author = re.sub('\r', '', name_author)
        file_code.append([name_author,book[1]])
file_code

[['Antonio_Machado_Páginas_escogidas', '49437'],
 ['Miguel_de_Unamuno_Abel_Sánchez__Una_Historia_de_Pasión', '44512'],
 ['Miguel_de_Unamuno_Amor_y_Pedagogía', '49149'],
 ['Miguel_de_Unamuno_Niebla_(Nivola)', '49836'],
 ['Miguel_de_Unamuno_La_Tía_Tula_(Novela)', '44358'],
 ['Miguel_de_Unamuno_Tres_novelas_ejemplares_y_un_prólogo', '55916'],
 ['Miguel_de_Unamuno_La_voz_de_la_conseja__t_1', '40827'],
 ['Ramón_del_Valle_Inclán_La_media_noche', '55448'],
 ['Ramón_del_Valle_Inclán_Romance_de_lobos__comedia_barbara', '10506'],
 ['Ramón_del_Valle_Inclán_Sonata_de_estío__memorias_del_marqués_de_Bradomín',
  '42424'],
 ['Ramón_del_Valle_Inclán_Sonata_de_otoño__Sonata_de_invierno__memorias_del_Marqués_de_Bradomín',
  '46182'],
 ['Ramón_del_Valle_Inclán_Sonata_de_primavera__memorias_del_marqués_de_Bradomín',
  '42440'],
 ['Manuel_Azaña_La_Biblia_en_España__Tomo_I_(de_3)', '51019'],
 ['Manuel_Azaña_La_Biblia_en_España__Tomo_II_(de_3)', '51020'],
 ['Manuel_Azaña_La_Biblia_en_España__Tomo_III_(de_3)'

Moving files to another folder

In [21]:
for file in file_code:
    os.system("mv gutenberg_CHANGE/files/" + file[1] + "/*.txt " + file[0] + ".txt")
    os.system("cp gutenberg_CHANGE/files/" + file[0] + ".txt " + "./DATA_filtered/" + file[0] + ".txt")